The dataset includes 12 months of data of the following tables:

1. subscription - holds the current state per each subscription 
1. order and order_item - an order and the line items for each order 
1. products - our sku table 
1. event_log - think of it as an activity log which tracks actions taken thru out the lifecycle of a subscription 
1. event_type - reference table for type_ID field (event_log table) 
 
Here are some questions we would like you to think about, some were deliberately left open ended to open the room for your data creativity and curiosity.  
1. Categorize the data set 
1. Can you make any hypothesis around subscription cancellations predictors 
1. We drive value for our clients by increasing subscription conversion, send as many shipments per subscription or use the IU (impulse upsell) feature to have customers add
one time items to their next order, based on that how else can you use this data to impact the above 
1. Which additional datasets would you like to see to further investigate the above? What new problems would you hope to solve or insights gleaned with this new data?


categorize the dataset:
cluster customers according to behaviors?
cluster items according to behaviors?
find clusters in the item-customer graph

predicting upsell opportunities (lower incentives; different products)
other datasets: clickstream data - categorize users by activity before they arrive at product page

first things first, get a clear definition of cancellation
basic stats on cancellation
hypothesize factors associated with cancellations - activities, engagement level/factors, product specific factors (aging out - e.g. diaper sizes)

easiest thing to do is subsample, choose from 1 store to limit noise from inventory factors - maybe even limit to 1 item at 1 store to start? -- group_by itemid, then perform some within-group analysis?

2. DQ issues. I probably should've asked for the mysql dump instead but I've gotten things mostly working now. However, compare the results from running wc on each file vs select count(*) from the resulting table:

```
     wc   select     diff
8829339  8457167  372,172  ./event_log_nulls.csv
     82       82        0  ./event_type_nulls.csv
6725109  6725109        0  ./order_item_nulls.csv
5396104  5396040       64  ./order_order_nulls.csv
6840406  6840385       21  ./product_product_nulls.csv
1938553  1935275     3278  ./subscription_subscription_nulls.csv
```

In the cast of the event logs, I think that a bunch of that error is accounted for by emails embedded in the email_html field which often span many lines in the raw data. Compare the following in which we grep for lines starting the right way:

    >> grep ^\"[0-9] event_log_nulls.csv | wc
    8457772 51181676 1149669198

This leaves a discrepancy of 603 rows.

I'd like to just forget about these entries since they are relatively rare in all tables. Unfortunately, we know that in the event_log table a lot of those will be errors resulting from the difficulty parsing the email html so the missing rows will not be a random sample and something similar could be happening in the other tables.

data problems / questions:

1. is there a data dictionary?
1. email data problems

Idea:
flow chart of subscriptions
Need:
nodes: (event_counter, event_type)
edges: num_people between each

No way to isolate individual subscriptions within event_log???

Customer id 0?

`offer_id` int(11) DEFAULT NULL, -- promo data?
`one_time` tinyint(1) NOT NULL,

only way to get incentives data: discount_total/sub_total in orders table??

timestamps are CST - OG server time.

### Cancellation Modeling questions
---


is cancellation a repeat-offender situation?

does non-modal shipping frequency impact likelihood of cancellation?

cancellation as a function of engagement? number of shipments as a function of engagement.
how does method-of-engagement affect this? autoship?

explore diaper and size stuff? look at cancel_reasons, that is a lot of info. "gift" in cancellation_reason -- maybe billing_name != shipping_name?

subscription, product, merchant factors?

rate of one-and-done subscription (ODS) for each merchant / product?

what factors are in common for the above? display list of ranked "worst for ODS" products and merchants

workflow
---

Build list of subscriptions with an ODS